# Limpando dados do OpenStreetMap com MongoDB

## Visão geral do projeto


Para realizar a limpeza dos dados, foi escolhida a área de Boston, EUA,  no https://www.openstreetmap.org e utilizado as técnicas de tratamento para avaliar a qualidade dos dados para validade, precisão, plenitude, consistência e uniformidade. Por último, foi escolhido MongoDB como modelo para armazenamento dos dados limpos, para, enfim, realizar análise exploratória nos dados.

Area do Mapa: Boston, EUA


https://www.openstreetmap.org/export#map=12/42.3677/-71.0458

## 1.  Auditoria de dados


In [1]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint
import codecs
import json
import os
from pymongo import MongoClient


osm_file = os.path.join("", "boston.osm")

# ================================================== #
#                Regex Expressions                   #
# ================================================== #
# somente contém letras minúsculas e válidas.
lower = re.compile(r'^([a-z]|_)*$')
# para tags válidas com dois pontos no valor.
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
# para tags com caracteres problemáticos.
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
# procura por padrões de escrita de endereços no final do texto
street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
# para códigos postais válidos
postal_code_re = re.compile(r'^[0-9]{5}(?:-[0-9]{4})?$', re.IGNORECASE)
# ================================================== #
#               Load Properties                    #
# ================================================== #
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Highway"]

mapping_street = { "Ave" :  "Avenue",
                    "Ave." : "Avenue",
                    "Ct" :   "Court",
                    "Hwy" :  "Highway",
                    "Park" : "Parkway",
                    "Pkwy" : "Parkway",
                    "Pl" :   "Place",
                    "Rd" :   "Road",
                    "Sq." :  "Square",
                    "St" :   "Street",
                    "st." :  "Street",
                    "ST" :   "Street",
                    "St" :   "Street",
                    "St." :  "Street",
                    "St," :  "Street",
                    "Cambrdige": "Cambridge",
                    "MA":    "Massachusetts"
                  }

# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file,  tags=('node', 'way', 'relation', 'tag'), verify_tags = False):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end':
            if verify_tags:
                if elem.tag in tags:
                    yield elem
                    root.clear()
            else:
                yield elem
                root.clear()

                
def print_sorted_dict(dictionary):
    keys = dictionary.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for key in keys:
        value = dictionary[key]
        print("%s: %d" % (key, value))
        
def audit_count_values(dictionary, value, pattern_regex):
    """Pesquisa os padrões de escrita e quantidade de ocorrência dos mesmos.
    
    Preenche o dicionário informado onde a chave será o padrão encontrado 
    e o valor será a soma das ocorrências encontradas.
    
    Args: 
        dictionary: Dicionário com os padrões de escrita.
        value:  Valor que será verificado.
    
    """
    m = pattern_regex.search(value)
    if m:
        value_type = m.group()
        dictionary[value_type] += 1
        
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def is_postal_code(elem):
    return elem.attrib['k'] == "addr:postcode"

def is_valid_tag(elem):
    """Verifica se a tag é válida.
    
    A tag só é válida se possuir os atributos K e V e se não possuir 
    caracteres inválidos no atributo K
    
    """
    value = problemchars.search(elem.get('k'))
    return value == None and 'k' in elem.attrib and 'v' in elem.attrib

 ####   1.1 Para ter um maior entendimento sobre os dados, realizei um processamento para descobrir quais e quantas tags existem no arquivo.

In [160]:
def count_tags(filename):
    """
    Percorre a ávore de elementos em buscas da quantidade de tags 
    que o arquivo possui.
    
    Args: 
        filename: O arquivo que será análisado.
    
    Returns: 
        Um dicionário com o nome da tag como chave e o total
        de registros encontrados.
        
        {'bounds': 1,
         'member': 36052,
         'meta': 1,
         'nd': 421429}
        
    """
    tags = {}
    for element in get_element(osm_file, verify_tags = False):
        if element.tag not in tags.keys():
            tags[element.tag] = 1
        else:
            tags[element.tag] += 1
    return tags
    

tags = count_tags(osm_file)
pprint.pprint(tags)

{'bounds': 1,
 'member': 36052,
 'meta': 1,
 'nd': 421429,
 'node': 335386,
 'note': 1,
 'osm': 1,
 'relation': 735,
 'tag': 187491,
 'way': 54602}


####    1.2 A função a seguir, tem a intenção de verificar se existe algum problema no valor de "k" para cada "tag" no arquivo. Para isso, será utilizado três expressões regulares para identificar padrões nos dados. 

In [18]:
def key_type(element, keys):
    """ Conta a quantidade de padrões encontrados em "K".
     
    Para cada valor encontrado em K, soma-se 1 ao seu 
    padrão correspondente.
    
    Args: 
        element: O elemento que será análisado.
        keys: dicionário com os padrões esperados.
    
    Returns: 
        Um dicionário com nome do padrão e o total de ocorrências encontradas.
        
        {'lower': 148488, 'lower_colon': 30911, 'other': 8092, 'problemchars': 0}
        
    """
    if lower.match(element.attrib['k']):
        keys['lower'] += 1
    elif lower_colon.match(element.attrib['k']):
        keys['lower_colon'] += 1
    elif problemchars.search(element.attrib['k']):
        keys['problemchars'] += 1
    else:
        keys['other'] += 1
    return keys



def process_map(filename):
    """ Processa os dados em busca de padrões no valor de "K".
     
    Define um dicionário com os padrões dos dados esperados em "K" 
    e percorre a ávore de elementos em buscas da quantidade de 
    itens que se encaixem nos padrões.
    
    Padrões dos valores esperados em K:
    
        - lower: somente contém letras minúsculas e válidas.
        - lower_colon: para tags válidas com dois pontos no valor.
        - problemchars: para tags com caracteres problemáticos.
        - other: para outras tags que não se enquadrem nas outras três
                 categorias.
    
    Args: 
        filename: O arquivo que será análisado.
    
    Returns: 
        Um dicionário com nome do padrão e o total de ocorrências encontradas.
        
        {'lower': 148488, 'lower_colon': 30911, 'other': 8092, 'problemchars': 0}
        
    """
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for element in get_element(osm_file, tags=('tag'), verify_tags = True):
        keys = key_type(element, keys)

    return keys

keys = process_map(osm_file)
pprint.pprint(keys)

{'lower': 148488, 'lower_colon': 30911, 'other': 8092, 'problemchars': 0}


#### 1.3 A próxima tarefa é identificar quantos usuários contribuíram com o mapa de Boston.

In [20]:
def get_user(element):
    """Recupera o usuário que contribuiu com o mapa.
    
    Args: 
        element: O elemento que será análisado.
    
    Returns: 
        O identificador do usuário encontrado
        
    """
    
    if element.get('uid'):
        uid = element.attrib["uid"]
        return uid
    else:
        return None


def process_map_user(filename):
    """ Processa os dados em busca dos usuários que contribuíram com os dados.
     
    Para para cada tag (node, way, relation) encontrada, recupera o usuário e adiona em um set.

    Args: 
        filename: O arquivo que será análisado.
    
    Returns: 
        Um set com os usuários encontrados.
        
        {'100042', '100049', '100054'}
        
    """
    users = set()
    for element in get_element(osm_file, tags=('node', 'way', 'relation'), verify_tags = True):
        if get_user(element):
             users.add(get_user(element))

    return users

users = process_map_user(osm_file)
print( '{} usuários contribuíram na região de Boston. '.format(len(users)))

931 usuários contribuíram na região de Boston. 


#### 1.4 A ideia da função a seguir é identificar os diferentes padrões de escrita dos nomes das ruas e quantas vezes ocorreram.

In [117]:
street_types = defaultdict(int)
 
def audit_street_occurrences(filename, pattern_regex):
    """Realiza auditoria nos padrões dos nomes das ruas cadastrados.
    
    
     Args: 
        filename: O arquivo que será análisado.
        pattern_regex: padrão que será pesquisado.
    """
    for elem in get_element(filename, tags=('tag'), verify_tags = True):
        if is_street_name(elem):
            audit_count_values(street_types, elem.attrib['v'], pattern_regex)    
    print_sorted_dict(street_types)
   

audit_street_occurrences(osm_file, street_type_re)

#1302: 1
#501: 1
1100: 1
1702: 1
1900: 1
3: 1
303: 1
500: 1
6: 1
Ave: 147
Ave.: 21
Avenue: 419
Boulevard: 7
Boylston: 1
Broadway: 33
Building: 1
Cambrdige: 2
Center: 8
Court: 7
Ct: 8
Drive: 55
Driveway: 1
Elm: 1
floor: 2
Garage: 1
Hall: 1
Hampshire: 1
Highway: 8
Hwy: 1
Lafayette: 1
Lane: 1
LEVEL: 1
Market: 1
Park: 16
Parkway: 2
Pkwy: 10
Pl: 1
Place: 55
Plaza: 3
Rd: 23
Road: 50
Row: 21
South: 1
Sq.: 1
Square: 49
St: 216
st: 1
ST: 1
St,: 1
St.: 32
Street: 1547
Terrace: 10
Way: 27
Wharf: 5
Windsor: 2
Winsor: 1
Yard: 1


#### 1.4 A próxima função realizará uma auditoria nos códigos postais a fim de identificar possíveis problemas.





In [13]:
postal_codes_types = {"Válidos": [], "Inválidos" : []}

def audit_postal_code(filename, pattern_regex):
    """Realiza auditoria nos padrões dos códigos postais cadastrados.
        
    Args: 
        filename: O arquivo que será análisado.
        pattern_regex: padrão que será pesquisado.
        
    """
    for elem in get_element(filename, tags=('tag'), verify_tags = True):
        if is_postal_code(elem):
            value = elem.get('v')
            if pattern_regex.match(value):
                postal_codes_types["Válidos"].append(value)
            else:
                postal_codes_types["Inválidos"].append(value)
                
    print('Foram encontrados {} códigos postais válidos'.format(len(postal_codes_types["Válidos"])))
    print('---------------------------------------------------------------------------------------')
    print('|Exemplo de códigos válidos encontrados {} '.format(postal_codes_types["Válidos"][0:5]))
    print('---------------------------------------------------------------------------------------')
    print('Foram encontrados {} códigos postais inválidos.'.format(len(postal_codes_types["Inválidos"])))
    print('---------------------------------------------------------------------------------------')
    print('|Exemplo de códigos inválidos encontrados {} '.format(postal_codes_types["Inválidos"]))
    print('---------------------------------------------------------------------------------------')

audit_postal_code(osm_file, postal_code_re)

Foram encontrados 1717 códigos postais válidos
---------------------------------------------------------------------------------------
|Exemplo de códigos válidos encontrados ['02110', '02114', '02116', '02116', '02116'] 
---------------------------------------------------------------------------------------
Foram encontrados 4 códigos postais inválidos.
---------------------------------------------------------------------------------------
|Exemplo de códigos inválidos encontrados ['MA', 'MA 02116', 'MA 02135', 'MA'] 
---------------------------------------------------------------------------------------


## 2 Problemas Encontrados.

Após coletar os dados do Open Street Map, resolvi realizar uma rápida auditoria nos dados, especificamente nos dados de endereço, e foi detectado dois problemas que precisam ser resolvidos antes de registrar os dados no MongoDB.
- Diversas abreviações para o mesmo endereço (Street = St, st, ST, St, St)
- Códigos postais inválidos ('MA', 'MA 02116', 'MA 02135', 'MA')

#### 2.1 Diversas abreviações para o mesmo endereço.

Para resolver os problemas de abreviações, foi necessário definir uma lista com os valores desejadas, para realizar a substituição dos valores problemáticos. Para isso, foi utilizada o bloco de código abaixo:

In [19]:
def audit_street_type(street_types, street_name):
    """Identifica os padrões de escrita dos endereços e separa em um dicinário. 
    
    Args: 
        street_types: Dicionário que separará os padrões encontrados.
        street_name: endereço que será categorizado.
        
    """
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def audit_street_standardization(osmfile):
    """Realiza auditoria nos padrões dos nomes das ruas cadastrados.
    
    Args: 
        osmfile: arquivo que será analisado.
        
    Returns: 
        Dicionário onde cada padrão encontrado será uma chave e o valor será um
        set dos endereços enquadrados no padrão.
        
    """
    street_types = defaultdict(set)
    for elem in get_element(osm_file, tags=('tag', 'way'), verify_tags = True):
        if 'k' in elem.attrib and is_street_name(elem):
            audit_street_type(street_types, elem.get('v'))
    
    return street_types


def update_name(name, mapping):
    """Atualiza o endereço com problema pelo valor padronizado.
    
    As abreviações encontradas serão substituídas pelos valores completos.
    
    Args: 
        name: endereço.
        mapping: dicionário com os valores desejados.
        
    Returns: 
        Se o endereço informado tiver uma abreviação, então terá seu valor 
        substituído pelo valor completo.
    
    """
    #remove valor indevido
    name = name.replace("#", "")
    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            if street_type in mapping.keys():
                name = re.sub(street_type_re, mapping[street_type], name)
    return name

st_types = audit_street_standardization(osm_file)

cont = 0
for st_type, ways in st_types.items():
    for name in ways:
        if cont == 15:
            break
        better_name = update_name(name, mapping_street)
        print( name, "=>", better_name)
        cont += 1

Somerville Ave => Somerville Avenue
Massachusetts Ave => Massachusetts Avenue
Commonwealth Ave => Commonwealth Avenue
Josephine Ave => Josephine Avenue
Highland Ave => Highland Avenue
Everett Ave => Everett Avenue
Francesca Ave => Francesca Avenue
Lexington Ave => Lexington Avenue
Morrison Ave => Morrison Avenue
Mystic Ave => Mystic Avenue
College Ave => College Avenue
Concord Ave => Concord Avenue
Willow Ave => Willow Avenue
Western Ave => Western Avenue
Massachusetts Ave. => Massachusetts Avenue


#### 2.2 Códigos postais inválidos.

Para entender melhor o problema, os códigos postais nos EUA possuem 2 formatos: com 5 dígitos(12345) e com 9 dígitos(12345-6789), mas nenhuma letra é permitida. Como encontramos códigos postais com letras, desenvolvemos a solução abaixo para a limpeza dos dados sujos.

In [17]:
def update_postal_code(value):
    """Atualiza o código postal com problema pelo valor padronizado.
    
    Args: 
        value: código postal.
        
    Returns: 
        Se o código postal informado estiver fora do padrão, então será ignorado
        e irá retornar None.
    
    """
    if postal_code_re.match(value):
        return value
    else:
        v = value.split()
        if len(v) > 1:
            return v[1]
        elif 'MA' in v[0]:
            return None
        else:
            return v[0]


def audit_fix_postal_code(filename, pattern_regex):
    elements = []
    for elem in get_element(filename, tags=('tag'), verify_tags = True):
        if is_postal_code(elem):
            elements.append(elem.get('v'))
    return elements

elements = audit_fix_postal_code(osm_file, postal_code_re)

for value in elements[0:15]:
    better_value = update_postal_code(value)
    print( value, "=>", better_value)  

02110 => 02110
02114 => 02114
02116 => 02116
02116 => 02116
02116 => 02116
02144 => 02144
02138 => 02138
02139 => 02139
02140 => 02140
02116 => 02116
02142 => 02142
02139 => 02139
02139 => 02139
02143 => 02143
02210 => 02210


## 3. Preparando-se para o Banco de Dados - MongoDB

Antes de importar os dados no MongoDB, precisamos realizar uma limpeza nos problemas encontrados na auditoria e estrutrar os dados de uma forma que facilite sua utilização posteriormente. Como o MongoDB trabalha com dados no formato JSON, será necessário converter de XML para JSON.

Os seguintes passos devem ser realizados:
- Processar apenas duas tags de nível superior: "node" and "way"
- Todos os atributos de "node" e "way" devem ser convertidos em dicionários, Exceto:
    - Os atributos de CREATED array deve ser adicionado sob a chave "created"
    - Os atributos de latitude e longitude deve ser adicionado no array "pos",
      para uso de indexação geoespacial. Certifique-se de que os valores são floats
      e não Strings. 
- Se a tag de segundo nível "k" possuir valores problemáticos, ela deve ser ignorada.
- Se a tag de segundo nível "k começar com "addr:", ela deve ser adicionada ao dicionário "address"
- Se a tag de segundo nível "k" não começar com "addr:", mas tiver ":", você pode processar ela da forma que se sentir melhor.
- Se tiver um segundo ":" que separa o tipo/direção de uma rua,  a tag deve ser ignorada, por exemplo:

< tag k="addr:housenumber" v="5158"/>

< tag k="addr:street" v="North Lincoln Avenue"/>

< tag k="addr:street:name" v="Lincoln"/>

< tag k="addr:street:prefix" v="North"/>

< tag k="addr:street:type" v="Avenue"/>

< tag k="amenity" v="pharmacy"/>

  Deve ser transformado em:

{...

    "address": {

    "housenumber": 5158,
    "street": "North Lincoln Avenue"
    
}

"amenity": "pharmacy",
...

}

- Para "way" especificamente:

  < nd ref="305896090"/> 
  
  < nd ref="1719825889"/>

Deve ser transformado em:

"node_refs": ["305896090", "1719825889"]


Ao final da limpeza e modelagem, os dados devem apresentar o seguinte formato:


{

    "id": "2406124091", 
    "type: "node",
    "visible":"true",
    "created": {
          "version":"2",
          "changeset":"17206049",
          "timestamp":"2013-08-03T16:43:42Z",
          "user":"linuxUser16",
          "uid":"1219059"
        },
    "pos": [41.9757030, -87.6921867],
    "address": {
          "housenumber": "5157",
          "postcode": "60625",
          "street": "North Lincoln Ave"
        },
    "amenity": "restaurant",
    "cuisine": "mexican",
    "name": "La Cabana De Don Luis",
    "phone": "1 (773)-271-5176"
}

In [23]:
CREATED = [ "version", "changeset", "timestamp", "user", "uid"]
ADDRESS = [ "housenumber", "postcode", "street"]

def shape_element(element):
    """Modela um elemento dentro do dicionário.
    
    Converte um elemento xml em um dicionário. 
    
    Returns: 
            Um dicionário com as informações estruturadas
    
    """
    node = {}
    if element.tag == "node" or element.tag == "way" :
        node["type"] = element.tag
        if "lat" in element.attrib and "lon" in element.attrib:
            node["pos"] = [float(element.get('lat')), float(element.get('lon'))]
            
        for attr in element.attrib:
            if attr in ['lat', 'lon']:
                continue
            elif attr in CREATED:
                if "created" not in node:
                    node["created"] = {}
                node["created"][attr] = element.get(attr)
            else:
                node[attr] = element.get(attr)
                
        address_dic = {}
        building_dic = {}
        mapper_second_level_elements(element, node, address_dic, building_dic)
        add_addres(address_dic, node)
        add_building(building_dic, node)
        
        return node
    else:
        return None

def add_addres(address_dic, node):
    """Adiciona dados de endereço no elemento principal.
    
    Caso existam dados de endereço nos elementos de segundo nível,
    estes serão adionados ao elemento principal como um atributo.
    
    Args: 
        address_dic: dicionário com dados de endereço
        node: dicinário que contém as informações do elemento.
    
    """
    if  len(address_dic) > 0:
        node["address"] = {}
        for key in ADDRESS:
            if key in address_dic:
                node["address"][key] = address_dic[key]

def add_building(building_dic, node):
    """Adiciona dados da construção no elemento principal.
    """
    if len(building_dic) > 0:
        node["building"] = building_dic
        
def mapper_second_level_elements(element, node, address_dic, building_dic):
    """Responsável por navegar entre as tags de segundo nível do elemento principal.
    
    Realiza iteração entre os elementos de segundo nível, a fim de estruturar e 
    armazenar suas informações.
    
    Args: 
        element: elemento de nível superior.
        node: dicinário que contém as informações do elemento.
        address_dic: dionário que armazenará os dados de endereço.
        building_dic: dicionário que armazenará os dados da construção.
    
    """
    for elem in element:
        if elem.tag == 'nd':
            mapper_nd(node, elem)
        if elem.tag == 'tag' and is_valid_tag(elem):
            mapper_tag(node, elem, address_dic, building_dic)
                
def mapper_tag(node, elem, address, building):
    """Extrai as informações da tag e separa de acordo com o tipo.
    
    Args: 
        elem: tag que terá seus dados extraídos.
        node: dicinário que contém as informações do elemento.
        address_dic: dionário que armazenará os dados de endereço.
        building_dic: dicionário que armazenará os dados da construção.
    
    """
    if "address" in elem.get('k'):
        return
    elif 'addr:' in elem.get('k'):
        mapper_addr(elem, address)
    elif 'building' in elem.get('k'):
        mapper_building(elem, building)
    else:
        node[elem.get('k')] = elem.get('v')
 

def mapper_building(elem, building):
    """Extrai as informações do elemento do tipo tag e com valor k ="building...
    
    Args: 
        elem: tag que terá seus dados extraídos.
        building_dic: dicionário que armazenará os dados da construção.
    
    """
    k = elem.get('k')
    v = elem.get('v')
    if 'building:' in k:
        k = k.replace('building:', '')
        building[k] = v
    else:
        building[k] = v

def mapper_addr(elem, address):
    """Extrai as informações do elemento do tipo tag e com valor k ="addr...
    
    Args: 
        elem: tag que terá seus dados extraídos.
        address: dicionário que armazenará os dados do endereço.
    
    """
    k = elem.get('k')
    v = elem.get('v')
    k = k.replace('addr:', '')
    if is_postal_code(elem):
        address[k] = update_postal_code(v)
    elif "street" in k:
        k = k.replace('street:', '')
        if k in ADDRESS:
            address[k] = update_name(v, mapping_street)
    else:
        if k in ADDRESS:
            address[k] = v

def mapper_nd(node, elem):
    """Extrai as informações do elemento do tipo nd.
    
    Args: 
        elem: tag que terá seus dados extraídos.
        node: dicionário que armazenará o valor de ref.
    
    """
    if "node_refs" not in node:
        node["node_refs"] = []
    if 'ref' in elem.attrib:
        node["node_refs"].append(elem.get("ref"))

def process_map(file_in, pretty = False):
    """Processa o mapa e converte de XML para JSON.
    
     Args: 
        file_in: mapa que será processado e convertido em JSON.
        pretty: Se True, irá adicionar espaços adionais no arquivo de saída
    
    """
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data


data = process_map(osm_file)
pprint.pprint(data[0:3])

[{'created': {'changeset': '106539',
              'timestamp': '2007-06-21T10:06:54Z',
              'uid': '8609',
              'user': 'ewedistrict',
              'version': '1'},
  'id': '30731187',
  'pos': [42.3788605, -71.039153],
  'type': 'node'},
 {'amenity': 'restaurant',
  'created': {'changeset': '39424017',
              'timestamp': '2016-05-19T13:06:19Z',
              'uid': '443130',
              'user': 'Alan Bragg',
              'version': '8'},
  'id': '31419556',
  'name': 'Firebrand Saints',
  'pos': [42.3624561, -71.0833314],
  'type': 'node'},
 {'created': {'changeset': '55231705',
              'timestamp': '2018-01-07T10:20:08Z',
              'uid': '165061',
              'user': 'mapper999',
              'version': '10'},
  'description': 'Outbound only',
  'id': '31419650',
  'operator': 'Massachusetts Bay Transportation Authority',
  'pos': [42.3627265, -71.0861277],
  'railway': 'subway_entrance',
  'type': 'node',
  'url': 'http://www.mbta.com/sch

## 4. Visão Geral dos Dados 

Esta seção demonstrará algumas estatísticas básicas sobre os dados, após importá-los no MongoDB.

In [48]:
client = MongoClient("mongodb://localhost:27017")
db = client.project

#### 4.1 Importando dados no MongoDB

In [101]:
data = []    
with open('boston.osm.json') as f:
    
    for line in f:
        data.append(json.loads(line))
        
db.streetmap.insert_many(data)
print( db.streetmap.find_one())

{'_id': ObjectId('5a6e344a18f0d32fb42da9a7'), 'type': 'node', 'pos': [42.3788605, -71.039153], 'id': '30731187', 'created': {'version': '1', 'timestamp': '2007-06-21T10:06:54Z', 'changeset': '106539', 'uid': '8609', 'user': 'ewedistrict'}}


#### 4.2 Tamanho dos arquivos

    boston.osm................ 76MB
    boston.osm.json........... 83.5MB


#### 4.3 Número de documentos

In [106]:
print("Existem {} documentos cadastrados".format(db.streetmap.find().count())) 

Existem 779976 documentos cadastrados


#### 4.4 Número de nós e caminhos

In [55]:
print("Existem {} nodes cadastrados".format(db.streetmap.find({"type" : "node"}).count()))
print("Existem {} ways cadastrados".format(db.streetmap.find({"type" : "way"}).count())) 

Existem 335305 nodes cadastrados
Existem 54591 ways cadastrados


#### 4.5 Número de usuários únicos

In [68]:
print("Existem {} usuários cadastrados".format(len(db.streetmap.distinct("created.user"))))

Existem 906 usuários cadastrados


#### 4.6 Número de dormitórios

In [71]:
print("Existem {} dormitórios cadastrados".format(db.streetmap.find({'building.building': "dormitory"}).count()))

Existem 49 dormitórios cadastrados


#### 4.7 Número de universidades

In [72]:
print("Existem {} universidades cadastradas".format(db.streetmap.find({'building.building': "university"}).count()))

Existem 175 universidades cadastradas


#### 4.8 Lista dos 10 usuários que mais contribuíram 

In [98]:
group = {"$group" : {"_id" : "$created.user", "count": {"$sum": 1}}}
sort  = {"$sort": {"count": -1}}
limit = {"$limit": 10}
pipeline = [group, sort, limit]

lista = db.streetmap.aggregate(pipeline)

pprint.pprint(list(lista))

[{'_id': 'crschmidt', 'count': 182410},
 {'_id': 'jremillard-massgis', 'count': 40533},
 {'_id': 'wambag', 'count': 26930},
 {'_id': 'morganwahl', 'count': 23147},
 {'_id': 'ryebread', 'count': 18951},
 {'_id': 'OceanVortex', 'count': 13009},
 {'_id': 'mapper999', 'count': 9411},
 {'_id': 'cspanring', 'count': 5687},
 {'_id': 'JasonWoof', 'count': 4651},
 {'_id': 'synack', 'count': 4211}]


#### 4.9 Lista das 10 origens que mais contribuíram

In [99]:
group = {"$group" : {"_id" : "$source", "count": {"$sum": 1}}}
sort  = {"$sort": {"count": -1}}
limit = {"$limit": 10}
pipeline = [group, sort, limit]

lista = db.streetmap.aggregate(pipeline)

pprint.pprint(list(lista))

[{'_id': None, 'count': 376919},
 {'_id': 'massgis_import_v0.1_20071008193615', 'count': 5595},
 {'_id': 'massgis_import_v0.1_20071008165629', 'count': 2282},
 {'_id': 'massdot_import_081211', 'count': 844},
 {'_id': 'massgis_import_v0.1_20071009093301', 'count': 769},
 {'_id': 'Bing', 'count': 748},
 {'_id': 'USGS Geonames', 'count': 333},
 {'_id': 'massgis_import_v0.1_20071013192438', 'count': 284},
 {'_id': 'massgis_import_v0.1_20071009094247', 'count': 282},
 {'_id': 'massgis_import_v0.1_20071008141127', 'count': 262}]


## 5. Outras ideias sobre os dados

Ao analisar os dados, me deparei com alguns problemas na estruturação e nos valores encontrados para alguns campos. O problema que chamou mais a atenção, foi o fato de que em alguns elementos existir a tag "address" com o agrupamento das informações de endereço e sem padrão algum. Eu recomendaria incentivar a não utilização dessa tag e estruturar os dados de endereço somente na tag "addr" como vimos em algumas entradas. Outra recomendação seria utilizar um xml com definição de tipos, a fim de evitar problemas com dados sujos. Pensando nas contribuições dos registros, podemos observar que 10 usuários contribuíram com cerca de 50% dos dados, como nem todas as pessoas tem conhecimento técnico para ficar enviando dados para atualizar os registros, eu recomendaria buscar parceria com redes sociais e outros aplicativos para atualizar os dados por meio de marcação das fotos nos locais ou recomendações.

1. Agrupamento das informações de endereço na tag "address". 
    - **Solução**: incentivar a não utilização dessa tag e estruturar os dados de endereço somente na tag "addr" como vimos em algumas entradas. 
        - **Benefícios:**
            - Facilidade na extração das informações de endereço nas tags estruturados.
        - **Problemas esperados:**
            - Revisão nos programas que realizavam extração das informações da tag "adress".
2. Tipos de dados incompativeis. Ex: código postal: 'MA 02116'        
    - **Solução:** utilizar um xml com definição de tipos.
        - **Benefícios:**
            - Dados mais padronizados e fáceis de processar
            - Eevitará problemas com dados sujos.
        - **Problemas esperados:**
            - Diminuição da flexibilidade de implementação dos programas.
            - Aumentará o tempo para o desenvolvimento das soluções.
3. Poucas usuários contribuem com a atualização dos dados:
    - **Solução:** buscar parcerias com redes sociais para a inclusão de dados no OSM.
        - **Benefícios:**
            - Aumento direto nas contribuições dos dados, devido ao grande número de usuários nas redes sociais.
            - Dados mais confiáveis.
        - **Problemas esperados:**
            - Melhoria nas infraestrutura dos servidores, devido a grande quantidade de requisições com origem nas redes sociais.
   


## 6. Conclusão

Neste projeto, escolhi analisar a área de Boston, EUA, e foi realizado o processo de limpeza de dados. Na fase de auditoria, nos encontrados alguns problemas como códigos postais inválidos e diversas abreviações diferentes para o mesmo endereço. Em seguida, realizamos as devidas correções e importamos os dados limpos no MongoDB. 
Para evitar este grande esforço para: auditar, limpar e resubmeter os dados, deveria padronizar a entrada de dados por Bots inteligentes ou por órgãos como o MassGIS - Escritório de Informação Geográfica e Ambiental da Commonwealth, a fim de garantir dados mais limpos que os inseridos manualmente.

## 7. Referências

https://stackoverflow.com/questions/1945920/why-doesnt-os-path-join-work-in-this-case

http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python

https://wiki.openstreetmap.org/wiki/MassGIS

https://stackoverflow.com/questions/28155857/mongodb-find-query-return-only-unique-values-no-duplicates

https://www.safaribooksonline.com/library/view/regular-expressions-cookbook/9781449327453/ch04s14.html